In [1]:
# data_generator.py

import numpy as np
import pandas as pd
import tensorflow as tf
from numpy.lib.stride_tricks import sliding_window_view
from dysts.flows import Lorenz

def simulate_lorenz(
    total_steps: int = 12000,
    burn_in: int = 200,
    dt: float = 0.01,
    sigma: float = 10.0,
    rho: float = 28.0,
    beta: float = 8/3
) -> np.ndarray:
    lorenz = Lorenz(dt=dt, sigma=sigma, rho=rho, beta=beta)
    _ = lorenz.make_trajectory(burn_in)
    return lorenz.make_trajectory(total_steps)

def build_windows(
    series: np.ndarray,
    window: int = 10,
    horizon: int = 1
) -> tuple[np.ndarray, np.ndarray]:
    T, D = series.shape
    N = T - window - horizon + 1
    sw = sliding_window_view(series, window_shape=(window, D))
    X = sw[:N, 0, :window, :]
    start = window + horizon - 1
    y = series[start : start + N]
    return X, y

def save_windows_to_csv(
    X: np.ndarray,
    y: np.ndarray,
    filename: str = "lorenz_windows.csv"
) -> None:
    N, window, D = X.shape
    X_flat = X.reshape(N, window * D)
    cols = [f"{feat}_t{t}" for t in range(window) for feat in ("x", "y", "z")]
    df = pd.DataFrame(X_flat, columns=cols)
    df[["x_next", "y_next", "z_next"]] = pd.DataFrame(y, columns=("x", "y", "z"))
    df.to_csv(filename, index=False)
    print(f"Saved {N} windows → {filename}")

def generate_and_save_csv(
    window: int = 20,
    horizon: int = 1,
    total_steps: int = 12000,
    burn_in: int = 200,
    dt: float = 0.01,
    csv_filename: str = "lorenz_windows.csv"
) -> None:
    """
    Full pipeline: simulate Lorenz, build windows, and save to CSV.
    No batching—one row per window+target.
    """
    series = simulate_lorenz(total_steps, burn_in, dt=dt)
    X, y   = build_windows(series, window, horizon)
    save_windows_to_csv(X, y, filename=csv_filename)

if __name__ == "__main__":
    # When you run `python data_generator.py`, this will execute:
    generate_and_save_csv(
        window=20,
        horizon=1,
        total_steps=12000,
        burn_in=200,
        dt=0.01,
        csv_filename="lorenz_windows.csv"
    )


Saved 11980 windows → lorenz_windows.csv
